# Multi timeframe strategy development

This notebook will show how to develop strategy using multiple timeframes with MLP Classificator.

## Steps:
1) Importing smallest dataframe, split to train and test resample to higher.
2) Featuring dataframe with indicators and other valuables informations (predictors).
3) Scale data, import MLPclassificator, create model, fit and predict.
4) Do steps 2 and 3 for every timeframe.
5) Merge predictions from higher tf to main one(smallest).
6) Vectorised backtest of predictions
7) Event based backtest
8) Adjusting sl, tp, managing trades to train set

### If there are good results in train set:

9) Backtest on test set(event based backtest with all params).

### Good results on test set?

10) Code strategy for MT5.
            

In [33]:
import pandas as pd
import numpy as np
from Features import *

## 1. Import, split and resample

In [34]:
df = pd.read_csv('ReadyData/EURUSD_2018_2023_15m.csv',index_col=0,parse_dates=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100393 entries, 2019-12-09 05:45:00 to 2023-12-19 18:15:00
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    100393 non-null  float64
 1   high    100393 non-null  float64
 2   low     100393 non-null  float64
 3   close   100393 non-null  float64
dtypes: float64(4)
memory usage: 3.8 MB


In [35]:
split = int(len(df) * 0.7)

train = df[:split]
test = df[split:]

print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70275 entries, 2019-12-09 05:45:00 to 2022-10-04 22:45:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    70275 non-null  float64
 1   high    70275 non-null  float64
 2   low     70275 non-null  float64
 3   close   70275 non-null  float64
dtypes: float64(4)
memory usage: 2.7 MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30118 entries, 2022-10-04 23:00:00 to 2023-12-19 18:15:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    30118 non-null  float64
 1   high    30118 non-null  float64
 2   low     30118 non-null  float64
 3   close   30118 non-null  float64
dtypes: float64(4)
memory usage: 1.1 MB
None


In [36]:
h1df = resample_dataframe(train,'60T')
h4df = resample_dataframe(train,'240T')
d1df = resample_dataframe(train,'1D')

## 2. Featuring datas and predictors

In [37]:
#df 15m tf
df['ret1'] = df['close'].pct_change()
df['ret5'] = df.ret1.rolling(5).sum()
df['std5'] = df.ret1.rolling(5).std()
df['H-L'] = df['high'] - df['low']
df['O-C'] = df['close'] - df['open']
df.dropna(inplace=True)

# For h1df
h1df['ret1'] = h1df['close'].pct_change()
h1df['ret5'] = h1df['ret1'].rolling(5).sum()
h1df['std5'] = h1df['ret1'].rolling(5).std()
h1df['H-L'] = h1df['high'] - h1df['low']
h1df['O-C'] = h1df['close'] - h1df['open']
h1df.dropna(inplace=True)

# For h4df
h4df['ret1'] = h4df['close'].pct_change()
h4df['ret5'] = h4df['ret1'].rolling(5).sum()
h4df['std5'] = h4df['ret1'].rolling(5).std()
h4df['H-L'] = h4df['high'] - h4df['low']
h4df['O-C'] = h4df['close'] - h4df['open']
h4df.dropna(inplace=True)

# For d1df
d1df['ret1'] = d1df['close'].pct_change()
d1df['ret5'] = d1df['ret1'].rolling(5).sum()
d1df['std5'] = d1df['ret1'].rolling(5).std()
d1df['H-L'] = d1df['high'] - d1df['low']
d1df['O-C'] = d1df['close'] - d1df['open']
d1df.dropna(inplace=True)


In [38]:
# Targets
df['retFut1'] = df.ret1.shift(-1)

# For h1df
h1df['retFut1'] = h1df['ret1'].shift(-1)

# For h4df
h4df['retFut1'] = h4df['ret1'].shift(-1)

# For d1df
d1df['retFut1'] = d1df['ret1'].shift(-1)


## 3. Scale data, create model,fit and predict

In [39]:
features_df = df[['H-L', 'O-C', 'ret5', 'std5']]
features_h1df = h1df[['H-L', 'O-C', 'ret5', 'std5']]
features_h4df = h4df[['H-L', 'O-C', 'ret5', 'std5']]
features_d1df = d1df[['H-L', 'O-C', 'ret5', 'std5']]

target_df = np.where(df.retFut1 > 0.0, 1, 0)
target_h1df = np.where(h1df.retFut1 > 0.0, 1, 0)
target_h4df = np.where(h4df.retFut1 > 0.0, 1, 0)
target_d1df = np.where(d1df.retFut1 > 0.0, 1, 0)

In [40]:

from sklearn.model_selection import train_test_split

train_length = int(len(df)*0.80)

X_train_df = features_df[:train_length]
X_test_df = features_df[train_length:]

y_train_df = target_df[:train_length]
y_test_df = target_df[train_length:]


train_length_h1 = int(len(h1df)*0.80)

# Splitting h1df into training and testing sets
X_train_h1df = h1df[:train_length_h1]
X_test_h1df = h1df[train_length_h1:]


train_length_h4 = int(len(h4df)*0.80)

# Splitting h4df into training and testing sets
X_train_h4df = h4df[:train_length_h4]
X_test_h4df = h4df[train_length_h4:]


train_length_d1 = int(len(d1df)*0.80)

# Splitting d1df into training and testing sets
X_train_d1df = d1df[:train_length_d1]
X_test_d1df = d1df[train_length_d1:]

# Assuming corresponding target variables for each dataframe are also sliced similarly
y_train_h1df = target_h1df[:train_length_h1]
y_test_h1df = target_h1df[train_length_h1:]

y_train_h4df = target_h4df[:train_length_h4]
y_test_h4df = target_h4df[train_length_h4:]

y_train_d1df = target_d1df[:train_length_d1]
y_test_d1df = target_d1df[train_length_d1:]

In [41]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


# Create the scaler model using train data
scaler.fit(X_train_df)

# Transform the training and test data using the scaler model created above
X_train = scaler.transform(X_train_df)
X_test = scaler.transform(X_test_df)


# Create the scaler model using train data for h1df
scaler.fit(X_train_h1df)
X_train_h1df_scaled = scaler.transform(X_train_h1df)
X_test_h1df_scaled = scaler.transform(X_test_h1df)

# Create the scaler model using train data for h4df
scaler.fit(X_train_h4df)
X_train_h4df_scaled = scaler.transform(X_train_h4df)
X_test_h4df_scaled = scaler.transform(X_test_h4df)

# Create the scaler model using train data for d1df
scaler.fit(X_train_d1df)
X_train_d1df_scaled = scaler.transform(X_train_d1df)
X_test_d1df_scaled = scaler.transform(X_test_d1df)


In [42]:
from sklearn.neural_network import MLPClassifier


In [44]:
seed = 42
# Create the MLPClassifier model
mlp_df = MLPClassifier(activation='logistic', hidden_layer_sizes=(
    5), random_state=seed, solver='sgd')


# Create MLPClassifier model for h1df
mlp_h1df = MLPClassifier(activation='logistic', hidden_layer_sizes=(5), random_state=seed, solver='sgd')

# Create MLPClassifier model for h4df
mlp_h4df = MLPClassifier(activation='logistic', hidden_layer_sizes=(5), random_state=seed, solver='sgd')

# Create MLPClassifier model for d1df
mlp_d1df = MLPClassifier(activation='logistic', hidden_layer_sizes=(5), random_state=seed, solver='sgd')

In [45]:
mlp_df.fit(X_train_df, y_train_df)

# Training MLPClassifier model for h1df
mlp_h1df.fit(X_train_h1df_scaled, y_train_h1df)

# Training MLPClassifier model for h4df
mlp_h4df.fit(X_train_h4df_scaled, y_train_h4df)

# Training MLPClassifier model for d1df
mlp_d1df.fit(X_train_d1df_scaled, y_train_d1df)

c:\Users\jakub\anaconda3\envs\Algo_vibes\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\jakub\anaconda3\envs\Algo_vibes\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\jakub\anaconda3\envs\Algo_vibes\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=5, random_state=42,
              solver='sgd')

In [ ]:
df['predicted_signal'] = mlp_df.predict(X)
data['strategy_returns_nn'] = data.retFut1 * data.predicted_signal